# Types

## 1. Schema

In [ ]:
import json


def pr(r):
    if r.errors:
        errors = [
            {
                "message": e.message, "locations": [
                    {
                        "line": lo.line, "column": lo.column
                    } for lo in e.locations
                ] if e.locations else []
            } for e in r.errors]
        s = json.dumps(errors, indent=4).replace("\n", "\n    ")
        return f"Error: {s}"

    s = json.dumps(r.data, indent=4).replace("\n", "\n    ")
    return f"Data: {s}"

### 1.1. Schema

A GraphQL Schema defines the types and relationships between Fields in your API.

A Schema is created by supplying the root `ObjectType` of each *operation*, *query(mandatory)*, *mutation* and *subscription*.

Schema will collect all type definitions related to the root operations and then supply them to the validator and executor.

In [ ]:
from graphene import (ObjectType, Schema, String, Mutation)


class RootQuery(ObjectType):
    root_field = String()
    other_field = String(name="_other_field_")

    def resolve_root_field(self, info):
        return "Root field"

    def resolve_other_field(self, info):
        return "Other field"


class RootMutation(Mutation):
    class Arguments:
        argument = String()

    field = String()

    def mutate(self, info, argument):
        ...


class RootSubscription(ObjectType):
    field = String()

    async def subscribe_field(self, info):
        ...


schema = Schema(
    query=RootQuery,
    # auto_camelcase=False,
    types=[],
    mutation=RootMutation,
    subscription=RootSubscription,
)

A Root Query is just a special `ObjectType` that defines the fields that are the entrypoint for your API. Root Mutation and Root Subscription are similar to Root Query, but for different operation types:

- Query fetches data
- Mutation changes data and retrieves the changes
- Subscription sends changes to clients in real-time

Review the GraphQL documentation on Schema for a brief overview of fields, schema and operations.

### 1.2. Querying

To query a schema, call the `execute` method on it. 

In [ ]:
q = """
    query queryRoot {
        rootField
        _other_field_
    }
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

: 

### 1.3. Types

There are some cases where the schema cannot access all of the types that we plan to have. For example, when a field returns an `Interface`, the schema doesn’t know about any of the implementations.

In this case, we need to use the `types` argument when creating the Schema.

In [ ]:
from graphene import (Interface, ID, String, List, ObjectType, Field)


class Person(Interface):
    id = ID(required=True)
    name = String(required=True)
    friends = List(lambda: Person)


class Student(ObjectType):
    class Meta:
        interfaces = (Person,)

    class_room = String(required=True)


class Query(ObjectType):
    class_leader = Field(Student, required=True)

    def resolve_class_leader(self, info):
        friend = Student(id=3, name="Lucy", class_room="2-1")
        return Student(id=1, name="Alvin", friends=[friend], class_room="1-1")


schema = Schema(
    query=Query,
    types=[Student]
)

q = """
    query findStudent {
        classLeader {
            id
            name
            friends {
                id
                name
                # classRoom  # The "friends" is "Person" type,
                             # cannot query by "class_room" field.
                             # If the "class_room" is necessary
                             # for "friends" field,
                             # change "friends = List(lambda: Person)"
                             # to "friends = List(lambda: Student)"
            }
            classRoom
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 1.4. Auto camelCase field names

By default all field and argument names (that are not explicitly set with the `name` arg) will be converted from `snake_case` to `camelCase` (as the API is usually being consumed by a **js/mobile** client)

For example with the `ObjectType`

In [ ]:
class Person(ObjectType):
    last_name = String()
    other_name = String(name="_other_name_")

the `last_name` field name is converted to `lastName`.

In case you don’t want to apply this transformation, provide a `name` argument to the field constructor. `other_name` converts to `_other_name_`(without further transformations).

Your query should look like

```graphql
{
    lastName
    _other_Name
}
```

To disable this behavior, set the `auto_camelcase` to False upon schema instantiation.

In [ ]:
schema = Schema(
    ...,
    auto_camelcase=False,
)

## 2. Scalars

Scalar types represent concrete values at the leaves of a query. There are several built in types that Graphene provides out of the box which represent common values in Python. You can also create your own Scalar types to better express values that you might have in your data model.

All Scalar types accept the following arguments. All are optional:

- `name`: *string*  
   Override the name of the Field.

- `description`: *string*  
   A description of the type to show in the GraphiQL browser.
   
- `required`: *boolean*  
   If True, the server will enforce a value for this field. See `NonNull`. Default is `False`.
   
- `deprecation_reason`: *string*  
   Provide a deprecation reason for the Field.
   
- `default_value`: *any*  
  Provide a default value for the Field.

### 2.1. Built in scalars

Graphene defines the following base Scalar Types that match the default GraphQL types:

#### `graphene.String`

Represents textual data, represented as UTF-8 character sequences. The String type is most often used by GraphQL to represent free-form human-readable text.

#### `graphene.Int`

Represents non-fractional signed whole numeric values. Int is a signed 32‐bit integer per the GraphQL spec

#### `graphene.Float`

Represents signed double-precision fractional values as specified by IEEE 754.

#### `graphene.Boolean`

Represents `true` or `false`.

#### `graphene.ID`

Represents a unique identifier, often used to refetch an object or as key for a cache. The ID type appears in a JSON response as a String; however, it is not intended to be human-readable. When expected as an input type, any string (such as “4”) or integer (such as 4) input value will be accepted as an ID.
Graphene also provides custom scalars for common values:

#### `graphene.Date`

Represents a Date value as specified by iso8601.

In [ ]:
from datetime import datetime, timedelta
from graphene import (Schema, ObjectType, Date)


class Query(ObjectType):
    one_week_from = Date(required=True, date=Date(required=True))

    def resolve_one_week_from(self, info, date):
        return date + timedelta(weeks=1)


schema = Schema(query=Query)
today = datetime.now().date()

q = f"""
    query {{
        oneWeekFrom(date: "{today}")
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### `graphene.DateTime`

Represents a DateTime value as specified by iso8601.

In [ ]:
from datetime import datetime as datetime_, timedelta
from graphene import Schema, ObjectType, DateTime


class Query(ObjectType):
    one_hour_from = DateTime(required=True, datetime=DateTime(required=True))

    def resolve_one_hour_from(self, info, datetime):
        return datetime + timedelta(hours=1)


schema = Schema(query=Query)
now = datetime_.now()

q = f"""
    query {{
        oneHourFrom(datetime: "{now.isoformat()}")
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### `graphene.Time`

Represents a Time value as specified by iso8601.

In [ ]:
from datetime import date, datetime, timedelta
from graphene import Schema, ObjectType, Time


class Query(ObjectType):
    one_hour_from = Time(required=True, time=Time(required=True))

    def resolve_one_hour_from(self, info, time):
        time = datetime.combine(date(1, 1, 1), time)
        return (time + timedelta(hours=1)).time()


schema = Schema(query=Query)
now = datetime.now().time()

q = f"""
    query {{
        oneHourFrom(time: "{now}")
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### `graphene.Decimal`

Represents a Python Decimal value.

In [ ]:
from decimal import Decimal as PyDecimal
from graphene import Schema, ObjectType, Decimal


class Query(ObjectType):
    add_one_to = Decimal(required=True, decimal=Decimal(required=True))

    def resolve_add_one_to(self, info, decimal):
        return decimal + PyDecimal("1")


schema = Schema(query=Query)
num = 10.50

q = f"""
    query {{
        addOneTo(decimal: "{num}")  # Decimal type must input as string
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### `graphene.JSONString`

Represents a JSON string.

In [ ]:
from graphene import Schema, ObjectType, JSONString, String


class Query(ObjectType):
    update_json_key = JSONString(
        required=True,
        json=JSONString(required=True),
        key=String(required=True),
        value=String(required=True)
    )

    def resolve_update_json_key(self, info, json, key, value):
        json[key] = value
        return json


schema = Schema(query=Query)

json_str = """{ \\"name\\": \\"Alvin\\" }"""
key = "gender"
value = "M"

q = f"""
    query {{
        updateJsonKey(json: "{json_str}", key: "{key}", value: "{value}")
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 2.2. Custom scalars

You can create custom scalars for your schema. The following is an example for creating a Base64 scalar:

In [ ]:
import base64 as b64
from typing import Any
from graphene import Schema, ObjectType, Scalar
from graphql.language import ast


class Base64(Scalar):
    @staticmethod
    def serialize(s: Any):
        """
        Serialize the result return from "resolve_xxx" function
        """
        print(f"  >> 's' is: {s}")
        return b64.b64encode(str(s).encode()).decode()

    @staticmethod
    def parse_literal(node):
        """
        Parse the literal from query string
        """
        print(f"  >> 'node' is: {node}")
        if isinstance(node, ast.StringValue):
            return Base64.parse_value(node.value)

    @staticmethod
    def parse_value(value):
        """
        Parse the value from query string
        """
        print(f"  >> 'value' is: {value}")
        return b64.b64decode(value.encode()).decode()


class Query(ObjectType):
    increment_encoded_id = Base64(
        required=True,
        base64=Base64(required=True)
    )

    def resolve_increment_encoded_id(self, info, base64):
        print(f"  > the value of 'base64' is: {base64}")
        return int(base64) + 1


schema = Schema(query=Query)
base64 = "NA=="
q = f"""
    query {{
        incrementEncodedId(base64: "{base64}")
    }}
"""

r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 2.3. Mounting Scalars

Scalars mounted in a `ObjectType`, `Interface` or `Mutation` act as `Field`s.

In [ ]:
from graphene import ObjectType, String, Field, Schema


class Query(ObjectType):
    name = String()  # pass the "String" instance
#   ~~~~~~~~~~~~~~~

    def resolve_name(self, info):
        return "Alvin"


schema = Schema(query=Query)

q = """
    query {
        name
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')


# Is equivalent to:

class Query(ObjectType):
    name = Field(String)  # pass the "String" type
#   ~~~~~~~~~~~~~~~~~~~~

    def resolve_name(self, info):
        return "Alvin"


schema = Schema(query=Query)

q = """
    query {
        name
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

> **Note:** when using the `Field` constructor directly, pass the type and not an instance.

Types mounted in a Field act as Arguments.

In [ ]:
from graphene import ObjectType, Field, String, Argument, Schema


class Query(ObjectType):
    name = Field(String, name=String(default_value="Alvin"))
#   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    def resolve_name(self, info, name):
        return f"Hello {name}"


schema = Schema(query=Query)

q = """
    query {
        name
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{r.data}"')


# Is equivalent to:

class Query(ObjectType):
    name = Field(String, name=Argument(String, default_value="Alvin"))
#   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    def resolve_name(self, info, name):
        return f"Hello {name}"


schema = Schema(query=Query)

q = """
    query {
        name
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

## 3. Lists and Non-Null

Object types, scalars, and enums are the only kinds of types you can define in Graphene. But when you use the types in other parts of the schema, or in your query variable declarations, you can apply additional type modifiers that affect validation of those values.

### 3.1. NonNull

In [ ]:
from graphene import Schema, ObjectType, NonNull, String, Argument


class Query(ObjectType):
    name = NonNull(String, value=Argument(String))
#   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    def resolve_name(self, info, value):
        return value if value else None


print("* Use NonNull(...):")

schema = Schema(query=Query)

for value in ("Alvin", ""):
    q = f"""
    query {{
        name(value: "{value}")
    }}
    """
    r = schema.execute(q)
    print(f'  the "r.data" of query "{q}" is: "{pr(r)}"')


# Is equivalent to:

class Query(ObjectType):
    name = String(value=String(), required=True)
#   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    def resolve_name(self, info, value):
        return value if value else None


print("* Use Field(required=True)")

schema = Schema(query=Query)

for value in ("Alvin", ""):
    q = f"""
    query {{
        name(value: "{value}")
    }}
    """
    r = schema.execute(q)
    print(f'  the "r.data" of query "{q}" is: "{pr(r)}"')

Here, we’re using a String type and marking it as Non-Null by wrapping it using the NonNull class. This means that our server always expects to return a non-null value for this field, and if it ends up getting a null value that will actually trigger a GraphQL execution error, letting the client know that something has gone wrong.

### 3.2. List

In [ ]:
from graphene import ObjectType, List, String, Schema


class Query(ObjectType):
    items = List(String, start=String(required=True), end=String(required=True))

    def resolve_items(self, info, start, end):
        return [chr(c) for c in range(ord(start), ord(end) + 1)]


schema = Schema(query=Query)

start = "A"
end = "F"

q = f"""
    query {{
        items(start: "{start}", end: "{end}")
    }}
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

Lists work in a similar way: We can use a type modifier to mark a type as a `List`, which indicates that this field will return a list of that type. It works the same for arguments, where the validation step will expect a list for that value.

### 3.3. NonNull Lists

By default items in a list will be considered nullable. To define a list without any nullable items the type needs to be marked as `NonNull`. For example:

In [ ]:
from graphene import ObjectType, List, NonNull, String


class Query(ObjectType):
    items = List(NonNull(String))

The above results in the type definition:

```graphql
type Character {
    appearsIn: [String!]
}
```

## 4. ObjectType

A Graphene ObjectType is the building block used to define the relationship between Fields in your Schema and how their data is retrieved.

The basics:

- Each ObjectType is a Python class that inherits from `graphene.ObjectType`.
- Each attribute of the ObjectType represents a `Field`.
- Each `Field` has a resolver method to fetch data (or Default Resolver).

### 4.1. Quick example

This example model defines a Person, with a first and a last name:

In [ ]:
from graphene import ObjectType, String, Field, Schema


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String()

    def resolve_full_name(self, info):
        return f'{self.first_name} {self.last_name}'

This ObjectType defines the field first_name, last_name, and full_name. Each field is specified as a class attribute, and each attribute maps to a Field. Data is fetched by our `resolve_full_name` resolver method for `full_name` field and the Default Resolver for other fields.

The above `Person` ObjectType has the following schema representation:

```graphql
type Person {
  firstName: String
  lastName: String
  fullName: String
}
```

### 4.2. Resolvers

A **Resolver** is a method that helps us answer **Queries** by fetching data for a **Field** in our **Schema**.

Resolvers are lazily executed, so if a field is not included in a query, its resolver will not be executed.

Each field on an *ObjectType* in Graphene should have a corresponding resolver method to fetch data. This resolver method should match the field name. For example, in the `Person` type above, the `full_name` field is resolved by the method `resolve_full_name`.

Each resolver method takes the parameters:

- **Parent Value Object (parent)** for the value object use to resolve most fields
- **GraphQL Execution Info (info)** for query and schema meta information and per-request context
- **GraphQL Arguments (\*\*kwargs)** as defined on the **Field**.

### 4.3. Resolver Parameters

#### 4.3.1. Parent Value Object (*parent*)

This parameter is typically used to derive the values for most fields on an *ObjectType*.

The first parameter of a resolver method (parent) is the value object returned from the resolver of the parent field. If there is no parent field, such as a root Query field, then the value for parent is set to the `root_value` configured while executing the query (default `None`). 

If we have a schema with Person type and one field on the root query.

In [ ]:
from collections import namedtuple
from graphene import ObjectType, String, Field, Schema


def get_human():
    PersonValue = namedtuple("Person", ["first_name", "last_name"])
    obj = PersonValue("Alvin", "Qu")
    print(f'  >> in "get_human", return object: "{obj}"id={id(obj)}')
    return obj


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String()

    def resolve_full_name(self, info):
        """
        This method can be describe as:

            @staticmethod
            def resolve_full_name(parent, info):
                ...

        The argument "self" is equals to "parent"

        "parent" argument is return value of "resolve_student"
        function in class "Query"
        """
        print(f'  >> in "resolve_full_name", '
              f'the "parent" object: "{self}"id={id(self)}')
        return f"{self.first_name} {self.last_name}"


class Query(ObjectType):
    student = Field(Person)

    def resolve_student(self, info):
        return get_human()  # return object as "student" field

When we execute a query against that schema.

In [ ]:
schema = Schema(query=Query)

q = """
    {
        student {
            firstName
            lastName
            fullName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

Then we go through the following steps to resolve this query:

- `parent` is set with the `root_value` from query execution (None).
- `Query.resolve_me` called with `parent` None which returns a value object `Person("Luke", "Skywalker")`.
- This value object is then used as `parent` while calling `Person.resolve_full_name` to resolve the scalar String value “Luke Skywalker”.
- The scalar value is serialized and sent back in the query response.


Each resolver returns the next **Parent Value Object (parent)** to be used in executing the following resolver in the chain. If the Field is a Scalar type, that value will be serialized and sent in the **Response**. Otherwise, while resolving Compound types like ObjectType, the value be passed forward as the next **Parent Value Object (parent)**.

This **Parent Value Object (parent)** is sometimes named `obj`, `parent`, or `source` in other GraphQL documentation. It can also be named after the value object being resolved (ex. `root` for a root Query or Mutation, and `person` for a Person value object). Sometimes this argument will be named `self` in Graphene code, but this can be misleading due to **Implicit staticmethod** while executing queries in Graphene.

The type of **Parent Value Object** can be any different type, for example:

In [ ]:
from graphene import ObjectType, String, Field, Schema


def get_human():
    return {
        "first_name": "Alvin",
        "last_name": "Qu"
    }


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String()

    def resolve_full_name(self, info):
        """
        This method can be describe as 

            @staticmethod
            def resolve_full_name(parent, info):
                ...

        The "parent" argument means return value from
        "resolve_student" method from "Query" class
        """
        return f"{self['first_name']} {self['last_name']}"


class Query(ObjectType):
    student = Field(Person)

    def resolve_student(self, info):
        """
        This method can be describe as

        @staticmethod
        def resolve_student(root, info):
            ...

        The argument "root" means "Query" class is the
        top level of "ObjectType"
        """
        return get_human()


schema = Schema(query=Query)

q = """
    {
        student {
            firstName
            lastName
            fullName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

The type of **Parent ObjectType** also be used for **ValueObject**, for example:

In [ ]:
from graphene import ObjectType, String, Field, Schema


def get_human():
    return Person(first_name="Alvin", last_name="Qu")


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String()

    def resolve_full_name(self, info):
        return f"{self.first_name} {self.last_name}"


class Query(ObjectType):
    student = Field(Person)

    def resolve_student(self, info):
        return get_human()


schema = Schema(query=Query)

q = """
    {
        student {
            firstName
            lastName
            fullName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

The GraphQL query field must conform to the definition of **Person ObjectType** and has nothing to do with the type returned by the resovle method of the parent class, for example: 

In [ ]:
from collections import namedtuple
from graphene import ObjectType, String, Field, Schema


def get_human():
    PersonValue = namedtuple("Person", ["first_name", "last_name"])
    return PersonValue("Alvin", "Qu")


class Person(ObjectType):
    full_name = String()

    def resolve_full_name(self, info):
        return f"{self.first_name} {self.last_name}"


class Query(ObjectType):
    student = Field(Person)

    def resolve_student(self, info):
        return get_human()


schema = Schema(query=Query)

q = """
    {
        student {
            # firstName
            # lastName
            fullName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### 4.3.2. GraphQL Execution Info (info)

The second parameter provides two things:

- reference to meta information about the execution of the current GraphQL Query (fields, schema, parsed query, etc.)
- access to per-request `context` which can be used to store user authentication, data loader instances or anything else useful for resolving the query.

Only context will be required for most applications. See **Context** for more information about setting context.

#### 4.3.3. GraphQL Arguments (**kwargs)

Any arguments that a field defines gets passed to the resolver function as keyword arguments. For example:

In [ ]:
from graphene import ObjectType, Field, String, Schema


def get_human(name):
    first_name, last_name = name.split(" ", 1)
    return Human(first_name=first_name, last_name=last_name)


class Human(ObjectType):
    first_name = String()
    last_name = String()


class Query(ObjectType):
    human_by_name = Field(Human, name=String(required=True))

    def resolve_human_by_name(self, info, name):
        return get_human(name=name)

You can then execute the following query:

```graphql
query {
    humanByName(name: "Luke Skywalker") {
        firstName
        lastName
    }
}
```

In [ ]:
schema = Schema(query=Query)

q = """
    {
        humanByName(name: "Alvin Qu") {
            firstName
            lastName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

*Note*: There are several arguments to a field that are “reserved” by Graphene. You can still define an argument that clashes with one of these fields by using the `args` parameter like so:

In [ ]:
from graphene import ObjectType, Field, String, Schema


class Query(ObjectType):
    answer = String(args={"description": String(required=True)})
    # answer = String(description=String(required=True))  # The "description" argument is "reserved" key word

    def resolve_answer(self, info, description):
        return f'The answer is: "{description}"'


schema = Schema(query=Query)

q = """
    query {
        answer(description: "Alvin")
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 4.4. Convenience Features of Graphene Resolvers

#### 4.4.1. Implicit staticmethod

One surprising feature of Graphene is that all resolver methods are treated implicitly as staticmethods. This means that, unlike other methods in Python, the first argument of a resolver is never `self` while it is being executed by Graphene. Instead, the first argument is always **Parent Value Object (parent)**. In practice, this is very convenient as, in GraphQL, we are almost always more concerned with the using the parent value object to resolve queries than attributes on the Python object itself.

The two resolvers in this example are effectively the same.

In [ ]:
from graphene import ObjectType, String, Field, Schema


class Person(ObjectType):
    first_name = String()
    last_name = String()

    @staticmethod
    def resolve_first_name(self, info):
        '''
        Decorating a Python method with `staticmethod` ensures that `self` will not be provided as an
        argument. However, Graphene does not need this decorator for this behavior.
        '''
        return self.first_name

    def resolve_last_name(self, info):
        '''
        Normally the first argument for this method would be `self`, but Graphene executes this as
        a staticmethod implicitly.
        '''
        return self.last_name


class Query(ObjectType):
    person = Field(Person, first_name=String(), last_name=String())

    def resolve_person(self, info, first_name, last_name):
        return Person(first_name=first_name, last_name=last_name)


schema = Schema(query=Query)

q = """
    query {
        person(firstName: "Alvin", lastName: "Qu") {
            firstName
            lastName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

If you prefer your code to be more explicit, feel free to use `@staticmethod` decorators. Otherwise, your code may be cleaner without them!

#### 4.4.2. Default Resolver

If a resolver method is not defined for a **Field** attribute on our *ObjectType*, Graphene supplies a default resolver.

If the **Parent Value Object (parent)** is a dictionary, the resolver will look for a dictionary key matching the field name. Otherwise, the resolver will get the attribute from the parent value object matching the field name.

In [ ]:
from collections import namedtuple
from graphene import ObjectType, String, Field, Schema


PersonValueObject = namedtuple("Person", ["first_name", "last_name"])


class Person(ObjectType):
    first_name = String()
    last_name = String()


class Query(ObjectType):
    me = Field(Person)
    my_best_friend = Field(Person)

    def resolve_me(self, info):
        # always pass an object for `me` field
        return PersonValueObject(first_name="Luke", last_name="Skywalker")

    def resolve_my_best_friend(self, info):
        # always pass a dictionary for `my_best_fiend_field`
        return {"first_name": "R2", "last_name": "D2"}


schema = Schema(query=Query)

q = """
    query {
        me {
            firstName
            lastName
        }
        myBestFriend {
            firstName
            lastName
        }
    }
"""
# With default resolvers we can resolve attributes from an object..
# With default resolvers, we can also resolve keys from a dictionary..
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 4.5. Advanced

#### 4.5.1. GraphQL Argument defaults


If you define an argument for a field that is not required (and in a query execution it is not provided as an argument) it will not be passed to the resolver function at all. This is so that the developer can differentiate between a `undefined` value for an argument and an explicit `null` value.

For example, given this schema:

In [ ]:
from graphene import ObjectType, String, Schema


class Query(ObjectType):
    hello = String(required=True, name=String())

    def resolve_hello(self, info, name):
        return name if name else 'World'

And this query:
    
```graphql
query {
    hello
}
```

An error will be thrown: `TypeError: resolve_hello() missing 1 required positional argument: 'name'`

In [ ]:
schema = Schema(query=Query)

q = """
    query {
        hello
    }
"""
r = schema.execute(q)
print(pr(r))

You can fix this error in several ways. Either by combining all keyword arguments into a dict:

In [ ]:
from graphene import ObjectType, String, Schema


class Query(ObjectType):
    hello = String(required=True, name=String())

    def resolve_hello(self, info, **kwargs):
        name = kwargs.get("name", "World")
        return f"Hello, {name}!"


schema = Schema(query=Query)

q = """
    query {
        hello
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

Or by setting a default value for the keyword argument:

In [ ]:
from graphene import ObjectType, String, Schema


class Query(ObjectType):
    hello = String(required=True, name=String())

    def resolve_hello(self, info, name="World"):
        return f"Hello, {name}!"


schema = Schema(query=Query)

q = """
    query {
        hello
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

One can also set a default value for an Argument in the GraphQL schema itself using Graphene!

In [ ]:
from graphene import ObjectType, String, Schema


class Query(ObjectType):
    hello = String(required=True, name=String(default_value="World"))

    def resolve_hello(self, info, name):
        return f"Hello, {name}!"


schema = Schema(query=Query)

q = """
    query {
        hello
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### 4.5.2. Resolvers outside the class

A field can use a custom resolver from outside the class:

In [ ]:
from graphene import ObjectType, String, Field, Schema


def resolve_full_name(instance, info):
    return f"{instance.first_name} {instance.last_name}"


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String(resolver=resolve_full_name)


def resolve_person(instance, info):
    return Person(first_name="Alvin", last_name="Qu")


class Query(ObjectType):
    person = Field(Person, resolver=resolve_person)


schema = Schema(query=Query)

q = """
    query {
        person {
            firstName
            lastName
            fullName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

#### 4.5.3. Instances as value objects

Graphene `ObjectTypes` can act as value objects too. So with the previous example you could use `Person` to capture data for each of the *ObjectType‘s* fields.

In [ ]:
from graphene import ObjectType


class Person(ObjectType):
    first_name = String()
    last_name = String()
    full_name = String()


peter = Person(first_name="Peter", last_name="Griffin")

print(f'* When "peter = Person(first_name="Peter", last_name="Griffin")"')
print(f'  the "peter.first_name" is: "{peter.first_name}"')
print(f'  the "peter.last_name" is: "{peter.last_name}"')
print(f'  the "peter.full_name" is: "{peter.full_name}"')

#### 4.5.4. Field camelcasing

Graphene automatically camelcases fields on *ObjectType* from `field_name` to `fieldName` to conform with GraphQL standards. 

### 4.6. *ObjectType* Configuration - Meta class

Graphene uses a Meta inner class on *ObjectType* to set different options.

#### 4.6.1. GraphQL type name

By default the type name in the GraphQL schema will be the same as the class name that defines the `ObjectType`. This can be changed by setting the `name` property on the `Meta` class:

In [ ]:
from graphene import ObjectType, String, Field, Schema


class MyGraphQlSong(ObjectType):
    class Meta:
        name = "Song"

    song_name = String(default_value="Hello song")


class Query(ObjectType):
    song = Field(MyGraphQlSong, default_value=MyGraphQlSong())


schema = Schema(query=Query)

q = """
    query {
        song {
            songName
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

## 5. Enums

An `Enum` is a special `GraphQL` type that represents a set of symbolic names (members) bound to unique, constant values.

### 5.1. Definition

You can create an `Enum` using classes:

In [ ]:
from graphene import Enum, ObjectType, String, Schema


class Episode(Enum):
    NEWHOPE = 4
    EMPIRE = 5
    JEDI = 6


class Movie(ObjectType):
    name = String()
    episode = Episode()  # Use "Enum" as Field


class Query(ObjectType):
    movie = Field(Movie)

    @staticmethod
    def resolve_movie(self, info):
        return Movie(name="New Hope", episode=Episode.NEWHOPE)


schema = Schema(query=Query)
q = """
    query {
        movie {
            name
            episode
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

But also using instances of Enum:

In [ ]:
from graphene import Enum, ObjectType, String, Field, Argument, Schema


Episode = Enum("Episode", [("NEWHOPE", 4), ("EMPIRE", 5), ("JEDI", 6)])


class Movie(ObjectType):
    name = String()
    episode = Field(Episode)


class Query(ObjectType):
    movie = Field(
        Movie,
        episode=Argument(Episode)  # use "Enum" as Argument
    )

    @staticmethod
    def resolve_movie(self, info, episode):
        episode = Episode.get(episode)  # Convert Enum value to Enum Object

        if episode == Episode.NEWHOPE:
            return Movie(name="New Hope", episode=episode)

        if episode == Episode.EMPIRE:
            return Movie(name="Empire", episode=episode)

        if episode == Episode.JEDI:
            return Movie(name="Jedi", episode=episode)

        raise ValueError("episode")


schema = Schema(query=Query)
q = """
    query {
        movie(episode: NEWHOPE) {
            name
            episode
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 5.2. Value descriptions

It's possible to add a description to an enum value, for that the enum value needs to have the `description` property on it.

In [ ]:
from graphene import Enum, ObjectType, String, Field, Schema, Argument


class Episode(Enum):
    NEWHOPE = 4
    EMPIRE = 5
    JEDI = 6

    @property
    def description(self):
        if self == Episode.NEWHOPE:
            return "New Hope Episode"

        return "Other Episode"


class EpisodeInfo(ObjectType):
    episode = Field(Episode)
    description = String()

    @staticmethod
    def resolve_description(self, info):
        return self.episode.description


class Query(ObjectType):
    episode_info = Field(EpisodeInfo, episode=Argument(Episode))

    @staticmethod
    def resolve_episode_info(self, info, episode):
        episode = Episode.get(episode)

        if episode is Episode.NEWHOPE:
            return EpisodeInfo(episode=episode)

        if episode is Episode.EMPIRE:
            return EpisodeInfo(episode=episode)

        if episode is Episode.JEDI:
            return EpisodeInfo(episode=episode)

        raise ValueError("episode")


schema = Schema(query=Query)
q = """
    query {
        episodeInfo(episode: NEWHOPE) {
            episode
            description
        }
    }
"""
r = schema.execute(q)
print(f'* The "r.data" of query "{q}" is: "{pr(r)}"')

### 5.3. Usage with Python Enums

In case the Enums are already defined it's possible to reuse them useing the `Enum.from_enum` function.

In [ ]:
from enum import Enum as PyEnum, auto
from graphene import Enum


class LegacyEnum(PyEnum):
    A = auto()
    B = auto()


NewEnum = Enum.from_enum(LegacyEnum)

print(f'* The "NewEnum.A" is: "{NewEnum.A}"')

### 5.4. Notes

`graphene.Enum` uses `enum.Enum` internally (or a backport if that's not avaliable) and can be used in a simmilar way, with the exception of memeber getters.

In the Python `Enum` implementation you can access a memeber by initing the Enum.

In [ ]:
from enum import Enum


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


print(f'* The "Color(1)" is: "{Color(1)}"')

However, in Graphene `Enum` you need to call get to have the same effect:

In [ ]:
from graphene import Enum


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


print(f'* The "Color.get(1)" is "{Color.get(1)}"')

## 6. Interfaces

An *Interface* is an abstract type that defines a certain set of fields that a type must include to implement the interface.

For example, you can define an Interface `Character` that represents any character in the Star Wars trilogy:

In [ ]:
from graphene import (
    Interface,
    ID,
    String,
    List,
    Field,
    ObjectType,
    Enum,
    Schema,
    Argument,
    Int
)


class Character(Interface):
    id = ID(required=True)
    name = String(required=True)
    friends = List(lambda: Character)   # In "Character" class, static field cannot use its owner class directly


class ShipType(Enum):
    CARGO_SHIP = 1
    BATTLE_SHIP = 2
    PASSENGER_SHIP = 3


class StarShip(ObjectType):
    name = String()
    ship_type = Field(ShipType)


class Human(ObjectType):
    class Meta:
        interfaces = (Character,)

    starships = List(StarShip)
    home_planet = String()


class Droid(ObjectType):
    class Meta:
        interfaces = (Character,)

    primary_function = String()

Both of these types have all of the fields from the `Character` interface, but also bring in extra fields, `home_planet`, `starships` and `primary_function`, that are specific to that particular type of character

The full GraphQL schema defition will look like this:

```graphql
interface Character {
    id: ID!
    name: String!
    friends: [Character]
}

enum ShipType {
    CARGO_SHIP
    BATTLE_SHIP
    PASSENGER_SHIP
}
    
type StarShip {
    name: String!
    shipType: ShipType!
}

type Human implements Character {
    id: ID!
    name: String!
    friends: [Character]
    starships: [StarShip]
    homePlanet: String
}

type Droid implements Character {
    id: ID!
    name: String!
    friends: [Character]
    primaryFunction: String!
}
```

Interfaces are useful when you want to return an object or set of objects, which might be of several different types.

For example, you can define a field `hero` that resolves to any `Character`, depending on the episode, like this:

In [ ]:
dataset = {
    "ships": [
        StarShip(
            name="Millennium Falcon",
            ship_type=ShipType.CARGO_SHIP
        )
    ],
    "humans": [
        Human(
            id=1,
            name="Luke Skywalker",
            home_planet="Tatooine"
        ),
        Human(
            id=2,
            name="Obi-Wan Kenobi",
            home_planet="Stewjon"
        ),
        Human(
            id=3,
            name="Han Solo",
            home_planet="Corellia"
        )
    ],
    "droids": [
        Droid(
            id=4,
            name="R2-D2",
            primary_function="Astronaut"
        ),
        Droid(
            id=5,
            name="3PO",
            primary_function="Etiquette"
        )
    ]
}

dataset["humans"][0].friends = [dataset["humans"][1], dataset["humans"][2], dataset["droids"][0], dataset["droids"][1]]
dataset["humans"][0].starships = [dataset["ships"][0]]
dataset["humans"][1].friends = [dataset["humans"][0]]
dataset["humans"][1].starships = [dataset["ships"][0]]
dataset["humans"][2].friends = [dataset["humans"][0], dataset["droids"][0], dataset["droids"][1]]
dataset["droids"][0].friends = [dataset["droids"][1], dataset["humans"][0], dataset["humans"][1]]
dataset["droids"][1].friends = [dataset["droids"][0], dataset["humans"][0], dataset["humans"][1]]


class HeroQuery(ObjectType):
    hero = Field(
        Character,
        required=True,
        episode=Argument(Int, required=True)
    )

    def resolve_hero(self, info, episode):
        if episode == 5:
            return dataset["humans"][0]
        return dataset["droids"][0]


class HumanQuery(ObjectType):
    human_hero = Field(
        Human,
        required=True,
        episode=Argument(Int, required=True)
    )

    def resolve_human_hero(self, info, episode):
        if episode == 5:
            return dataset["humans"][0]

        return dataset["humans"][2]


class DroidQuery(ObjectType):
    droid_hero = Field(
        Droid,
        required=True,
        episode=Argument(Int, required=True)
    )

    def resolve_droid_hero(self, info, episode):
        if episode == 5:
            return dataset["droids"][0]

        return dataset["droids"][1]


class Query(HeroQuery, HumanQuery, DroidQuery):
    pass


# schema = Schema(query=Query, types=[Human, Droid])
schema = Schema(query=Query)

q = """
    query($episode: Int!) {
        hero(episode: $episode) {
            id
            name
            friends {
                id
                name
            }
        }
    }"""
v = {"episode": 5}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

q = """
    query {
        hero(episode: 6) {
            id
            name
            friends {
                id
                name
            }
        }
    }"""
v = {"episode": 6}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

q = """
    query($episode: Int!) {
        humanHero(episode: $episode) {
            id
            name
            friends {
                id
                name
            }
            starships {
                name
                shipType
            }
        }
    }"""
v = {"episode": 5}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

q = """
    query($episode: Int!) {
        droidHero(episode: $episode) {
            id
            name
            friends {
                id
                name
            }
            primaryFunction
        }
    }"""
v = {"episode": 5}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}" is: "{pr(r)}"')

This allows you to directly query for fields that exist on the Character interface as weel as selecting specific fields on any type that implements the interface using inline fragments.

For example, the following query:

```graphql
{
    query HeroForEpisode($episode: Int!) {
        hero(episode: $episode) {
            __typename
            name
            ... on Droid {
                primaryFunction
            }
            ... on Human {
                homePlanet
            }
        }
    }
}
```

Will return the following data with variables `{"episode": 4}`:

```json
{
    "data": {
        "hero": {
            "__typename": "Droid",
            "name": "R2-D2",
            "primaryFunction": "Astromech"
        }
    }
}
```

And different data with the variables `{"episode": 5}`:

```json
{
    "data": {
        "hero": {
            "__typename": "Human",
            "name": "Luke Skywalker",
            "homePlanet": "Tatooine"
        }
    }
}
```

In [ ]:
q = """
    query($episode: Int!) {
        hero(episode: $episode) {
            __typename
            name
            ... on Droid {
                primaryFunction
            }
            ... on Human {
                homePlanet
            }
        }
    }"""

v = {"episode": 5}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

v = {"episode": 6}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

### 6.1. Resolving data objects to types

As you build out your schema in Graphene it's common for your resolvers to return objects that represent the data backing your GraphQL rather than instances of the Graphene types (e.g. Diango or SQLAlchemy models). This works well with `ObjectType` and `Scalar` fields, however when you start using interfaces you might come across this error:

```python
"Abstract type Character must resolve to an Object type at runtime for field Query.hero ..."
```

This happens because Graphene doesn't have enought information to convert the data object into a Graphene type needed to resolve the `Interface`. To solve this you can define a `resolve_type` class method on the `Interface` which maps a data object to Graphene type:

In [ ]:
from graphene import Interface, ID, String


class Character(Interface):
    id = ID(required=True)
    name = String(required=True)

    @classmethod
    def resolve_type(cls, instance, info):
        if instance.type == "DROID":
            return Droid
        return Human

## 7. Unions

Union types are very similar to interfaces, but they don't get to specify any common fields between the types.

The basics:

- Each Union is a Python class that inherits from `graphene.Union`
- Unions don't have any fields on it, just links to the possible object types.

### 7.1. Quick example

This example model defines several ObjectTypes whith their own fields. `SearchResult` is the implementation of `Union` of this object types.

In [ ]:
from graphene import ObjectType, String, Int, Union


class Human(ObjectType):
    name = String()
    born_in = String()


class Droid(ObjectType):
    name = String()
    primary_function = String()


class StarShip(ObjectType):
    name = String()
    length = Int()


class SearchResult(Union):
    class Meta:
        types = (Human, Droid, StarShip)

Wherever we return a `SearchResult` type in our schema, we might get a `Human`, a `Droid`, or a `StarShip`. Note that members of *union type* need to be concrete object types; you can't create a *union types* out of *interfaces* or other *unions*.

The above types have the following representation in a schema:

```graphql
type Droid {
  name: String
  primaryFunction: String
}

type Human {
  name: String
  bornIn: String
}

type Ship {
  name: String
  length: Int
}

union SearchResult = Human | Droid | Starship 
```

## 8. Mutations

A `Mutations` is a special `ObjectType` that also defines an input.

### 8.1. Quick example

This example defines a `Mutations`:

In [ ]:
from graphene import Mutation, String, Int, Boolean, Field, Schema


class Person(ObjectType):
    name = String()
    age = Int()


class CreatePerson(Mutation):
    class Arguments:
        name = String()

    ok = Boolean()
    person = Field(lambda: Person)

    def mutate(self, info, name):
        person = Person(name=name, age=40)
        ok = True
        return CreatePerson(ok=ok, person=person)

**person** and **ok** are the output fields of the `Mutations` when it is resolved.

**Arguments** attributes are the arguments that the *Mutations* `CreatePerson` needs for resolving, in this case **name** will be the only argument for the mutation.

**mutate** is the funciton that will be applied once the mutation is called. This method is just a special resolver that we can change data within. it tasks the same arguments as the standard query **Resolver Parameters**.

So, we can finish out schema like this:

In [ ]:
class PersonMutations(ObjectType):
    create_person = CreatePerson.Field()


class Query(ObjectType):
    person = Field(Person)


schema = Schema(query=Query, mutation=PersonMutations)

### 8.2. Executing the Mutation

Then, if we query (`schema.execute(query_str)`) the following:

```graphql
mutation {
    createPerson(name: "Peter") {
        person {
            name
        }
        ok
    }
}
```

We should receive:

```json
{
    "createPerson": {
        "person" : {
            "name": "Peter"
        },
        "ok": true
    }
}
```

In [ ]:
q = """
    mutation($name: String) {
        createPerson(name: $name) {
            person {
                name
                age
            }
            ok
        }
    }"""
v = {"name": "Alvin"}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

### 8.3. InputFields and InputObjectTypes

`InputFields` are used in *mutations* to allow nested input data for *mutations*

To use an `InputField` you define an `InputObjectType` that specifies the structure of your input data

In [ ]:
from graphene import InputObjectType, String, Int, Mutation, Field


class Person(ObjectType):
    name = String()
    age = Int()


class PersonInput(InputObjectType):
    name = String(required=True)
    age = Int(required=True)


class CreatePerson(Mutation):
    class Arguments:
        person_data = PersonInput(required=True)

    person = Field(Person, required=True)

    def mutate(self, info, person_data):
        person = Person(
            name=person_data.name,
            age=person_data.age
        )
        return CreatePerson(person=person)

Note that **name** and **age** are part of **person_data** now

Using the above *mutation* your new query would look like this:

In [ ]:
class PersonMutation(ObjectType):
    create_person = CreatePerson.Field()


schema = Schema(mutation=PersonMutation)

q = """
    mutation($personData: PersonInput!) {
        createPerson(personData: $personData) {
            person {
                name
                age
            }
        }
    }"""
v = {"personData": {"name": "Alvin", "age": 40}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

**InputObjectTypes** can also be fields of **InputObjectTypes** allowing you to have as complex of input data as you need

In [ ]:
from graphene import InputObjectType, Field, Float, String, InputField, Mutation, ID, Schema


class LatLngType(ObjectType):
    lat = Float()
    lng = Float()


class LocationType(ObjectType):
    id = ID()
    name = String()
    latlng = Field(LatLngType)


class LatLngInput(InputObjectType):
    lat = Float()
    lng = Float()


# A location has a latlng associated to it
class LocationInput(InputObjectType):
    name = String()
    latlng = InputField(LatLngInput)


class LocationCreate(Mutation):
    class Arguments:
        location_data = LocationInput(required=True)

    location = Field(LocationType, required=True)

    def mutate(self, info, location_data):
        return LocationCreate(
            location=LocationType(
                id=101,
                name=location_data.name,
                latlng=LatLngType(
                    lat=location_data.latlng.lat,
                    lng=location_data.latlng.lng
                )
            )
        )


class LocationUpdate(Mutation):
    class Arguments:
        id = ID(required=True)
        location_data = LocationInput(required=True)

    location = Field(LocationType, required=True)

    def mutate(self, info, id, location_data):
        return LocationUpdate(
            location=LocationType(
                id=id,
                name=location_data.name,
                latlng=LatLngType(
                    lat=location_data.latlng.lat,
                    lng=location_data.latlng.lng
                )
            )
        )


class LocationMutation(ObjectType):
    location_create = LocationCreate.Field()
    location_update = LocationUpdate.Field()


schema = Schema(mutation=LocationMutation)

q = """
    mutation($locationInput: LocationInput!) {
        locationCreate(locationData: $locationInput) {
            location {
                id
                name
                latlng {
                    lat
                    lng
                }
            }
        }
    }"""
v = {"locationInput": {"name": "Test", "latlng": {"lat": 11.12, "lng": 22.34}}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

q = """
    mutation($id: ID!, $locationInput: LocationInput!) {
        locationUpdate(id: $id, locationData: $locationInput) {
            location {
                id
                name
                latlng {
                    lat
                    lng
                }
            }
        }
    }"""
v = {"id": 202, "locationInput": {"name": "Test", "latlng": {"lat": 11.12, "lng": 22.34}}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

### 8.4. Output type example

To return an existing `ObjectType` instead of a *mutation-specific type*, set the Output attribute to the desired `ObjectType`:

In [ ]:
from graphene import ObjectType, ID, String, Int, InputObjectType, Mutation, Schema


class PersonMixin:
    name = String(required=True)
    age = Int()


class PersonType(ObjectType, PersonMixin):
    id = ID(required=True)


class PersonInput(InputObjectType, PersonMixin):
    pass


class PersonCreate(Mutation):
    class Arguments:
        person_input = PersonInput(required=True)

    Output = PersonType

    def mutate(self, info, person_input):
        return PersonType(
            id=101,
            name=person_input.name,
            age=person_input.age
        )


class PersonUpdate(Mutation):
    class Arguments:
        id = ID(required=True)
        person_input = PersonInput(required=True)

    Output = PersonType

    def mutate(self, info, id, person_input):
        return PersonType(
            id=id,
            name=person_input.name,
            age=person_input.age
        )


class PersonMutation(ObjectType):
    person_create = PersonCreate.Field()
    person_update = PersonUpdate.Field()


schema = Schema(mutation=PersonMutation)

q = """
    mutation($personInput: PersonInput!) {
        personCreate(personInput: $personInput) {
            __typename
            id
            name
            age
        }
    }"""
v = {"personInput": {"name": "Alvin", "age": 40}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

q = """
    mutation($id: ID!, $personInput: PersonInput!) {
        personUpdate(id: $id, personInput: $personInput) {
            __typename
            id
            name
            age
        }
    }"""
v = {"id": 202, "personInput": {"name": "Alvin", "age": 40}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')